In [1]:
import json
import pandas as pd
import pickle

In [7]:
obs_answer = pickle.load(open("data/obs_examples/obs_answering.pkl", "rb"))
obs_asking = pickle.load(open("data/obs_examples/obs_asking.pkl", "rb"))
obs_guessing = pickle.load(open("data/obs_examples/obs_guessing.pkl", "rb"))

In [141]:
obs_answer

kaggle_environments.utils.Struct

In [9]:
len(obs_answer['questions']), len(obs_answer['answers'])

(20, 19)

In [140]:
obs_answer.guesses

['Beach',
 'Paris',
 'Paris',
 'Paris',
 'Berlin',
 'Berlin',
 'Berlin',
 'Belgium',
 'Belgrade',
 'Vienna',
 'Belgrade',
 'Belgrade',
 'Vatican City',
 'Vatican City',
 'Warsaw',
 'Krakow',
 'Wroclaw',
 'Wroclaw',
 'Wroclaw']

In [11]:
obs_guessing

{'remainingOverageTime': 300,
 'questions': ['Is it a place?',
  'Is it in Europe?',
  'Is it a capital city?',
  'Is it a city in the UK?',
  'Is it a city in France?',
  'Is it a city in Italy?',
  'Is it a city in Spain?',
  'Is it a city in Germany?',
  'Is it a city in Portugal?',
  'Is it a city in Greece?',
  'Is it a city in Austria?',
  'Is it a city in Switzerland?',
  'Is it a city in Belgium?',
  'Is it a city in Ireland?',
  'Is it a city in Poland?',
  'Is it Warsaw?',
  'Is it Krakow?',
  'Is it Gdansk?',
  'Is it Poznan?',
  'Is it Wroclaw?'],
 'guesses': ['Beach',
  'Paris',
  'Paris',
  'Paris',
  'Berlin',
  'Berlin',
  'Berlin',
  'Belgium',
  'Belgrade',
  'Vienna',
  'Belgrade',
  'Belgrade',
  'Vatican City',
  'Vatican City',
  'Warsaw',
  'Krakow',
  'Wroclaw',
  'Wroclaw',
  'Wroclaw'],
 'answers': ['yes',
  'yes',
  'yes',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'yes',
  'no',
  'no',
  'no',
  'no',
  'no'],


In [31]:
from data.keywords import KEYWORDS_JSON
keywords_json = json.loads(KEYWORDS_JSON)
keywords_flat = []
for category in keywords_json:
    for keyword in category['words']:
        keywords_flat.append(
            {
                "keyword": keyword['keyword'].lower(),
                "alts": keyword['alts'],
                "category": category['category'],
            }
        )

In [32]:
from data.keywords_things import KEYWORDS_JSON
keywords_json = json.loads(KEYWORDS_JSON)
for category in keywords_json:
    for keyword in category['words']:
        keywords_flat.append(
            {
                "keyword": keyword['keyword'].lower(),
                "alts": keyword['alts'],
                "category": category['category'],
            }
        )

In [35]:
# ",".join([k['keyword'] for k in keywords_flat if k['category'] == "things"])

In [109]:
keywords_csv = pd.read_csv("data/keywords.csv").drop(["Unnamed: 0"], axis=1)
keywords_csv.keyword = keywords_csv.keyword.apply(str.lower)

In [110]:
set_csv = set(keywords_csv.keyword.apply(str.lower))
set_json = set([k['keyword'] for k in keywords_flat])

In [111]:
# Cool, we have all keyworkds in CSV
# But categories are merged into place (city, country, landmark)

In [112]:
# Generate from keywords_flat, keyword -> category
keyword_category_dict = {
    k['keyword'].lower(): k['category']
    for k in keywords_flat
}

In [113]:
# Replace category from keyword_category_dict
# If no key - take existing category value
keywords_csv['category'] = keywords_csv.apply(
    lambda x: keyword_category_dict.get(x.keyword, x.category),
    axis=1
)

In [114]:
keywords_csv.category.value_counts()

category
things      1496
city         314
country      188
landmark      48
Name: count, dtype: int64

In [124]:
# sampled_df = keywords_csv.groupby('category').apply(lambda x: x.sample(n={
#         "things": 100,
#         "city": 40,
#         "country": 20,
#         "landmark": 40,
#     }.get(x.name, 0)
# )).reset_index(drop=True)
# category_dict = {category: sampled_df[sampled_df['category'] == category] for category in sampled_df['category'].unique()}
sampled_df = keywords_csv.groupby('category').apply(lambda x: x).reset_index(drop=True)
category_dict = {category: sampled_df[sampled_df['category'] == category] for category in sampled_df['category'].unique()}

C:\Users\HaveToCook\AppData\Local\Temp\ipykernel_34380\203393168.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = keywords_csv.groupby('category').apply(lambda x: x).reset_index(drop=True)


In [125]:
category_dict.keys()

dict_keys(['city', 'country', 'landmark', 'things'])

In [128]:
category_dict['city']

,keyword,category
0,acapulco mexico,city
1,adelaide australia,city
2,algiers algeria,city
3,amsterdam netherlands,city
4,anaheim california,city
...,...,...
309,zeeland netherlands,city
310,zhongshan china,city
311,zincantan mexico,city
312,zunyi china,city


In [127]:
city_to_alt_dict = {}
for keyword in keywords_flat:
    if not keyword['alts']:
        continue
    if not keyword['category'] == "city":
        continue
    print(keyword)
    city_to_alt_dict[keyword['keyword']] = keyword['alts'][0]

{'keyword': 'amsterdam netherlands', 'alts': ['amsterdam', 'amsterdam holland'], 'category': 'city'}
{'keyword': 'anaheim california', 'alts': ['anaheim', 'anaheim ca', 'anaheim usa'], 'category': 'city'}
{'keyword': 'austin texas', 'alts': ['austin', 'austin tx', 'austin usa'], 'category': 'city'}
{'keyword': 'auckland new zealand', 'alts': ['auckland', 'auckland nz'], 'category': 'city'}
{'keyword': 'asheville north carolina', 'alts': ['asheville', 'asheville nc', 'asheville usa'], 'category': 'city'}
{'keyword': 'ashgabat turkmenistan', 'alts': ['ashgabat', 'ashgabad turkmenistan'], 'category': 'city'}
{'keyword': 'athens greece', 'alts': ['athens'], 'category': 'city'}
{'keyword': 'athens georgia', 'alts': ['athens', 'athens ga', 'athens usa'], 'category': 'city'}
{'keyword': 'atlanta georgia', 'alts': ['atlanta', 'atlanta ga', 'atlanta usa'], 'category': 'city'}
{'keyword': 'antwerp belgium', 'alts': ['antwerp'], 'category': 'city'}
{'keyword': 'adelaide australia', 'alts': ['adel

In [132]:
# Replace bullshit Cairo Egypt with just Cairo
category_dict['city'].copy()['keyword'] = category_dict['city'].keyword.apply(lambda x: city_to_alt_dict.get(x, x))

In [133]:
pickle.dump(category_dict, open("keywords_by_category.pkl", "wb"))

In [134]:
category_dict['city']

,keyword,category
0,acapulco,city
1,adelaide,city
2,algiers,city
3,amsterdam,city
4,anaheim,city
...,...,...
309,zeeland,city
310,zhongshan,city
311,zincantan,city
312,zunyi,city
